## Что потребуется установить 
- Docker: https://docs.docker.com/engine/install/
- Установить NVIDIA Container Toolkit: https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html
- Образы triton infernece server: https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver/tags


## Установка NVIDIA Container Toolkit

In [ ]:
curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list

In [ ]:
sudo apt-get update

In [ ]:
sudo apt-get install -y nvidia-container-toolkit

## Установка Triton Inference Server

In [ ]:
docker 

## 1. Базовые понятия triton infernece server

## Ансамбль моделей

```bash
# Expected folder layout
model_repository/
├── text_detection
│   ├── 1
│   │   └── model.onnx
│   └── config.pbtxt
└── text_recognition
    ├── 1
    │   └── model.onnx
    └── config.pbtxt
```

**config.pbtxt**
``` text proto
name: "text_detection"
backend: "onnxruntime"
max_batch_size : 256
input [
  {
    name: "input_images:0"
    data_type: TYPE_FP32
    dims: [ -1, -1, -1, 3 ]
  }
]
output [
  {
    name: "feature_fusion/Conv_7/Sigmoid:0"
    data_type: TYPE_FP32
    dims: [ -1, -1, -1, 1 ]
  }
]
output [
  {
    name: "feature_fusion/concat_3:0"
    data_type: TYPE_FP32
    dims: [ -1, -1, -1, 5 ]
  }
]
```

Разберем файл конфигов
- name - Необязательно поле, которое обозначает название модели (Note: Название должно совпадать с названием каталога!)
- backend/platform - указывает на бэкенд, который используется для запуска модели. Бэкенды, которые поддерживает triton: [Документация](https://docs.nvidia.com/deeplearning/triton-inference-server/archives/triton_inference_server_220/user-guide/docs/backend.html), [Github](https://github.com/triton-inference-server/backend#backends)
- max_batch_sizу - Значение, которое указывает на максимальный размер батча, который воспринимает модель
- input and output:

На примере выше, размерность входных данных соотвествует при max_batch_size > 0:
```
[-1] + dims = [-1, -1, -1, -1, 3]
```
Для моделей с max_batch size = 0:
```
dims = [-1, -1, -1, 3]
```